Ниже будет код для генерации изображения по полученным файлам:
weather && chickpea 

In [1]:
# Создание датасета
import math
import pandas as pd
import numpy as np
import csv
from PIL import Image


columns = ['doy', 'geo_id', 'gr_covar1', 'gr_covar2', 'gr_covar3', 'gr_covar4', 'gr_covar5', 'gr_covar6', 'gr_covar7', 'gr_covar8', 'gr_covar9', 'gr_covar10', 'gr_covar11', 'gr_covar12', 'gr_covar13', 'gr_covar14', 'gr_covar15', 'gr_covar16', 'gr_covar17', 'gr_covar18', 'response', 'species', 'year']

w_columns = []

for i in range(25):
    w_columns.append('dl%d'%(i+1))
    w_columns.append('rain%d'%(i+1))
    w_columns.append('srad%d'%(i+1))
    w_columns.append('tmax%d'%(i+1))
    w_columns.append('tmin%d'%(i+1))
    
for w in w_columns:
    columns.append(w)

data = {column: [] for column in columns}
df = pd.DataFrame(data)

In [2]:
def get_weather_information(filename):
    df = pd.read_csv(filename)
    weather_info = []
    weather_groups = df.columns.values.tolist()
    # ['month', 'doy', 'geo_id', 'dl', 'rain', 'srad', 'tmax', 'tmin', 'year']
    for group in weather_groups:
        tmp = []
        for dset in df[group]:
            tmp.append(dset)
        weather_info.append(np.array(tmp))
    weather_info = np.array(weather_info).transpose()
    return weather_info


def get_chickpea_information(filename):
    df = pd.read_csv(filename)
    plant_groups = df.columns.values.tolist()
    chickpea_info = []
    # ['doy', 'geo_id', 'gr_covar', 'response', 'species', 'year']
    for group in plant_groups:
        tmp = []
        for dset in df[group]:
            if group == 'gr_covar1':
                if (dset == 1.):
                    dset = 2
            elif group == 'gr_covar4':
                if (dset == 1.):
                    dset = 2
            elif group == 'gr_covar7':
                if (dset == 1.):
                    dset = 2
            elif group == 'gr_covar10':
                if (dset == 1.):
                    dset = 2
            elif group == 'gr_covar13':
                if (dset == 1.):
                    dset = 2
            elif group == 'gr_covar16':
                if (dset == 1.):
                    dset = 2
            elif group == 'gr_covar2':
                if (dset == 1.):
                    dset = 0
            elif group == 'gr_covar5':
                if (dset == 1.):
                    dset = 0
            elif group == 'gr_covar8':
                if (dset == 1.):
                    dset = 0
            elif group == 'gr_covar11':
                if (dset == 1.):
                    dset = 0
            elif group == 'gr_covar14':
                if (dset == 1.):
                    dset = 0
            elif group == 'gr_covar17':
                if (dset == 1.):
                    dset = 0
            elif group == 'gr_covar3':
                if (dset == 1.):
                    dset = 1
            elif group == 'gr_covar6':
                if (dset == 1.):
                    dset = 1
            elif group == 'gr_covar9':
                if (dset == 1.):
                    dset = 1
            elif group == 'gr_covar12':
                if (dset == 1.):
                    dset = 1
            elif group == 'gr_covar15':
                if (dset == 1.):
                    dset = 1
            elif group == 'gr_covar18':
                if (dset == 1.):
                    dset = 1
            tmp.append(dset)
        chickpea_info.append(np.array(tmp))
    chickpea_info = list(map(list, zip(*chickpea_info)))
    plants = []
    for chickpea in chickpea_info:
        plants.append(chickpea)
    return plants

In [3]:
def get_data(filename_chickpea, filename_weather, df, columns): 
    tmp_weather_first = get_weather_information(filename_weather)
    weather = []
    for i in range(len(tmp_weather_first)):
        weather.append(tmp_weather_first[i])
    all_chickpea = get_chickpea_information(filename_chickpea)
    for plant in all_chickpea:
        data = plant
        days_data=[]
        
        for day in weather:
            if (day[-1] == plant[-1]) and (day[2] == plant[1]) and (plant[0] - 5 <= day[1] < plant[0] + 20):
                for i in range(3, 8):
                    days_data.append(day[i])
                    
        data = data + days_data
        data = {column: [datum] for column, datum in zip(columns, data)}
        tmp = pd.DataFrame(data)
        df = pd.concat([df, tmp], axis=0)
        
    return df        
                    

chickpea = "chickpea-ssm-snp.csv"
weather = "weather.csv"
df = get_data(chickpea, weather, df, columns)

In [4]:
def convert_weather_value_to_px(left_border, value):
    if isinstance(value, float):
        value *= 100
        left_border *= 100
        left_border, value = int(left_border), int(value)
    b = 0 if value >= 0 else 255
    shifted_value = abs(value) - left_border
    r = shifted_value // 256
    g = shifted_value % 256
    return [0, 0, r, g]


def convert_snp_to_px(placed, value, bits):
    r, g, b = 0, 0, 0
    if value == 1:
        r = (1 << placed) | bits[0]
        g = bits[1]
    elif value == 0:
        r = bits[0]
        g = bits[1]
    elif value == 2:
        r = (1 << placed) | bits[0]
        g = (1 << placed) | bits[1]
    bits = [r, g]
    return [math.sqrt(r) if math.sqrt(r) <= 255 else 255, math.sqrt(g) if math.sqrt(g) <= 255 else 255], bits


def transform(df, path_to_save, csv_class):
    num_of_image = 1
    idx1 = [i for i in range(2, 20)]
    idx2 = [i for i in range(23, 148)]
    with open(csv_class, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for i in range(df.shape[0]):
            pixels1 = []
            pixels2 = []
            snp_pixels = []
            placed = 0
            temp_bit = [0, 0]
            for c in columns[2:20]:
                temp_array, temp_bit = convert_snp_to_px(placed, df.iloc[i:i+1,idx1][c][0], temp_bit)
                snp_pixels.append(temp_array)
                placed += 1
            index_snp = 0
            repeat_5 = 1
            day_pixels = []
            for w in w_columns:
                tmp = convert_weather_value_to_px(0, df.iloc[i:i+1,idx2][w][0])
                tmp[0] = snp_pixels[index_snp][0]
                tmp[1] = snp_pixels[index_snp][1]
                day_pixels.append(tmp)
                if index_snp > 16:
                    index_snp = 0
                index_snp += 1
                if repeat_5 % 5 == 0:
                    pixels1.append(day_pixels)
                    pixels2.append(day_pixels)
                    day_pixels = []
                repeat_5 = repeat_5 + 1
            pixels = np.zeros((25, 25, 4))
            for i in range(25):
                for j in range(25):
                    pixels[i][j] = pixels1[i][j % 5]
            pixels = np.array(pixels, dtype=np.uint8)
            new_image = Image.fromarray(pixels, mode='RGBA').rotate(90, expand=True)
            new_image.save(path_to_save + "image" + str(num_of_image) + ".png")
            print("image" + str(num_of_image) + ".png")
            writer.writerow(["image" + str(num_of_image) + ".png", str(df.iloc[i:i+1,[21]])])  # only response
            num_of_image += 1

path_to_save = "images/"
csv_response = "response.csv"
transform(df, path_to_save, csv_response)

image1.png
image2.png
image3.png
image4.png
image5.png
image6.png
image7.png
image8.png
image9.png
image10.png
image11.png
image12.png
image13.png
image14.png
image15.png
image16.png
image17.png
image18.png
image19.png
image20.png
image21.png
image22.png
image23.png
image24.png
image25.png
image26.png
image27.png
image28.png
image29.png
image30.png
image31.png
image32.png
image33.png
image34.png
image35.png
image36.png
image37.png
image38.png
image39.png
image40.png
image41.png
image42.png
image43.png
image44.png
image45.png
image46.png
image47.png
image48.png
image49.png
image50.png
image51.png
image52.png
image53.png
image54.png
image55.png
image56.png
image57.png
image58.png
image59.png
image60.png
image61.png
image62.png
image63.png
image64.png
image65.png
image66.png
image67.png
image68.png
image69.png
image70.png
image71.png
image72.png
image73.png
image74.png
image75.png
image76.png
image77.png
image78.png
image79.png
image80.png
image81.png
image82.png
image83.png
image84.png
i

image641.png
image642.png
image643.png
image644.png
image645.png
image646.png
image647.png
image648.png
image649.png
image650.png
image651.png
image652.png
image653.png
image654.png
image655.png
image656.png
image657.png
image658.png
image659.png
image660.png
image661.png
image662.png
image663.png
image664.png
image665.png
image666.png
image667.png
image668.png
image669.png
image670.png
image671.png
image672.png
image673.png
image674.png
image675.png
image676.png
image677.png
image678.png
image679.png
image680.png
image681.png
image682.png
image683.png
image684.png
image685.png
image686.png
image687.png
image688.png
image689.png
image690.png
image691.png
image692.png
image693.png
image694.png
image695.png
image696.png
image697.png
image698.png
image699.png
image700.png
image701.png
image702.png
image703.png
image704.png
image705.png
image706.png
image707.png
image708.png
image709.png
image710.png
image711.png
image712.png
image713.png
image714.png
image715.png
image716.png
image717.png

image1254.png
image1255.png
image1256.png
image1257.png
image1258.png
image1259.png
image1260.png
image1261.png
image1262.png
image1263.png
image1264.png
image1265.png
image1266.png
image1267.png
image1268.png
image1269.png
image1270.png
image1271.png
image1272.png
image1273.png
image1274.png
image1275.png
image1276.png
image1277.png
image1278.png
image1279.png
image1280.png
image1281.png
image1282.png
image1283.png
image1284.png
image1285.png
image1286.png
image1287.png
image1288.png
image1289.png
image1290.png
image1291.png
image1292.png
image1293.png
image1294.png
image1295.png
image1296.png
image1297.png
image1298.png
image1299.png
image1300.png
image1301.png
image1302.png
image1303.png
image1304.png
image1305.png
image1306.png
image1307.png
image1308.png
image1309.png
image1310.png
image1311.png
image1312.png
image1313.png
image1314.png
image1315.png
image1316.png
image1317.png
image1318.png
image1319.png
image1320.png
image1321.png
image1322.png
image1323.png
image1324.png
image1

image1841.png
image1842.png
image1843.png
image1844.png
image1845.png
image1846.png
image1847.png
image1848.png
image1849.png
image1850.png
image1851.png
image1852.png
image1853.png
image1854.png
image1855.png
image1856.png
image1857.png
image1858.png
image1859.png
image1860.png
image1861.png
image1862.png
image1863.png
image1864.png
image1865.png
image1866.png
image1867.png
image1868.png
image1869.png
image1870.png
image1871.png
image1872.png
image1873.png
image1874.png
image1875.png
image1876.png
image1877.png
image1878.png
image1879.png
image1880.png
image1881.png
image1882.png
image1883.png
image1884.png
image1885.png
image1886.png
image1887.png
image1888.png
image1889.png
image1890.png
image1891.png
image1892.png
image1893.png
image1894.png
image1895.png
image1896.png
image1897.png
image1898.png
image1899.png
image1900.png
image1901.png
image1902.png
image1903.png
image1904.png
image1905.png
image1906.png
image1907.png
image1908.png
image1909.png
image1910.png
image1911.png
image1

image2429.png
image2430.png
image2431.png
image2432.png
image2433.png
image2434.png
image2435.png
image2436.png
image2437.png
image2438.png
image2439.png
image2440.png
image2441.png
image2442.png
image2443.png
image2444.png
image2445.png
image2446.png
image2447.png
image2448.png
image2449.png
image2450.png
image2451.png
image2452.png
image2453.png
image2454.png
image2455.png
image2456.png
image2457.png
image2458.png
image2459.png
image2460.png
image2461.png
image2462.png
image2463.png
image2464.png
image2465.png
image2466.png
image2467.png
image2468.png
image2469.png
image2470.png
image2471.png
image2472.png
image2473.png
image2474.png
image2475.png
image2476.png
image2477.png
image2478.png
image2479.png
image2480.png
image2481.png
image2482.png
image2483.png
image2484.png
image2485.png
image2486.png
image2487.png
image2488.png
image2489.png
image2490.png
image2491.png
image2492.png
image2493.png
image2494.png
image2495.png
image2496.png
image2497.png
image2498.png
image2499.png
image2

image3016.png
image3017.png
image3018.png
image3019.png
image3020.png
image3021.png
image3022.png
image3023.png
image3024.png
image3025.png
image3026.png
image3027.png
image3028.png
image3029.png
image3030.png
image3031.png
image3032.png
image3033.png
image3034.png
image3035.png
image3036.png
image3037.png
image3038.png
image3039.png
image3040.png
image3041.png
image3042.png
image3043.png
image3044.png
image3045.png
image3046.png
image3047.png
image3048.png
image3049.png
image3050.png
image3051.png
image3052.png
image3053.png
image3054.png
image3055.png
image3056.png
image3057.png
image3058.png
image3059.png
image3060.png
image3061.png
image3062.png
image3063.png
image3064.png
image3065.png
image3066.png
image3067.png
image3068.png
image3069.png
image3070.png
image3071.png
image3072.png
image3073.png
image3074.png
image3075.png
image3076.png
image3077.png
image3078.png
image3079.png
image3080.png
image3081.png
image3082.png
image3083.png
image3084.png
image3085.png
image3086.png
image3

image3602.png
image3603.png
image3604.png
image3605.png
image3606.png
image3607.png
image3608.png
image3609.png
image3610.png
image3611.png
image3612.png
image3613.png
image3614.png
image3615.png
image3616.png
image3617.png
image3618.png
image3619.png
image3620.png
image3621.png
image3622.png
image3623.png
image3624.png
image3625.png
image3626.png
image3627.png
image3628.png
image3629.png
image3630.png
image3631.png
image3632.png
image3633.png
image3634.png
image3635.png
image3636.png
image3637.png
image3638.png
image3639.png
image3640.png
image3641.png
image3642.png
image3643.png
image3644.png
image3645.png
image3646.png
image3647.png
image3648.png
image3649.png
image3650.png
image3651.png
image3652.png
image3653.png
image3654.png
image3655.png
image3656.png
image3657.png
image3658.png
image3659.png
image3660.png
image3661.png
image3662.png
image3663.png
image3664.png
image3665.png
image3666.png
image3667.png
image3668.png
image3669.png
image3670.png
image3671.png
image3672.png
image3

image4188.png
image4189.png
image4190.png
image4191.png
image4192.png
image4193.png
image4194.png
image4195.png
image4196.png
image4197.png
image4198.png
image4199.png
image4200.png
image4201.png
image4202.png
image4203.png
image4204.png
image4205.png
image4206.png
image4207.png
image4208.png
image4209.png
image4210.png
image4211.png
image4212.png
image4213.png
image4214.png
image4215.png
image4216.png
image4217.png
image4218.png
image4219.png
image4220.png
image4221.png
image4222.png
image4223.png
image4224.png
image4225.png
image4226.png
image4227.png
image4228.png
image4229.png
image4230.png
image4231.png
image4232.png
image4233.png
image4234.png
image4235.png
image4236.png
image4237.png
image4238.png
image4239.png
image4240.png
image4241.png
image4242.png
image4243.png
image4244.png
image4245.png
image4246.png
image4247.png
image4248.png
image4249.png
image4250.png
image4251.png
image4252.png
image4253.png
image4254.png
image4255.png
image4256.png
image4257.png
image4258.png
image4